In this tutorial, we'll demonstrate how to use OpenAI functioncalling.  We'll use the GPT-3.5 model to identify and call a function in the context of a conversation. We will use a simple function that returns a string as a response.

Let's import the required libraries

In [3]:
# Importing the necessary library for OpenAI API
import os
import openai
openai.api_base = "https://openai.vocareum.com/v1"
# Importing the necessary library for JSON used in OpenAI API calls
import json 

# Define your OpenAI API key 
api_key = os.environ["OPENAI_API_KEY"]
openai.api_key = api_key

#### Step 1: Define the Function

First, let's define a simple Python function that takes a string as an argument and returns a string as a response.

In [4]:
def simple_function(input_string):
    return f"Function called with argument: {input_string}"

#### Step 2: Define the Tools

Next, we define the `tools` list. This list contains the functions that the assistant can use. Each function is described by its name, description, and parameters.

In [5]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "simple_function",
            "description": "A simple function that returns a string",
            "parameters": {
                "type": "object",
                "properties": {
                    "input_string": {
                        "type": "string",
                        "description": "A string to pass to the function"
                    }
                },
                "required": ["input_string"]
            }
        }
    }
]

#### Step 3: Define the Initial Conversation Messages

We define the initial conversation messages. The first message is from the system, describing the role of the assistant. The second message is from the user, requesting the function call. Feel free to change the argument!

In [6]:
messages = [
    {"role": "system", "content": "You are an assistant that can call a simple function."},
    {"role": "user", "content": "Call the simple function with the argument 'Hello there, World!'."}
]

#### Step 4: Generate a Response from the Model

We call `openai.ChatCompletion.create()` (for openai < 1.0) or `openai.chat.completions.create()` (for openai > 1.0) to generate a response from the GPT-3.5 model. The model, messages, and tools are passed as arguments. The `tool_choice` is set to "auto", allowing the model to choose which tool (function) to use.

In [7]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    tools=tools,
    tool_choice="auto",
)

In [8]:
# Christoph:
print(response)

{
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "annotations": [],
        "content": null,
        "refusal": null,
        "role": "assistant",
        "tool_calls": [
          {
            "function": {
              "arguments": "{\"input_string\":\"Hello there, World!\"}",
              "name": "simple_function"
            },
            "id": "call_yheqGtkayvzUW1ZWdpd2hFPH",
            "type": "function"
          }
        ]
      }
    }
  ],
  "created": 1743180438,
  "id": "chatcmpl-BG79SnDdLk6iKj7C8h9yksE7XTKdF",
  "model": "gpt-3.5-turbo-0125",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 20,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens": 82,
    "prompt_tokens_det

#### Step 5: Extract the Tool Calls and Call the Function

We extract any tool calls from the response. 

- If there are any tool calls, we map the tool names to their corresponding Python functions. 
- For each tool call in the response, we extract the function name and call the corresponding Python function with the arguments provided by the model.

In [9]:
tool_calls = response.choices[0].message.tool_calls

if tool_calls:
    available_functions = {
        "simple_function": simple_function,
    }

    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)

        if function_name == 'simple_function':
            function_response = function_to_call(
                input_string=function_args.get("input_string"),
            )

        messages.append({
            "role": "assistant",
            "content": function_response,
        })

#### Step 6: Print the Conversation

Finally, we print the conversation messages to see the function call and response in the context of the conversation.

In [10]:
for message in messages:
    print(f"{message['role']}: {message['content']}")

system: You are an assistant that can call a simple function.
user: Call the simple function with the argument 'Hello there, World!'.
assistant: Function called with argument: Hello there, World!
